This notebook allows you to manually and interactively to estimate FFH from a given GSV pano image, which is useful for ground-truth data assessment and updates (if required) and more reliable than visual assessment. It requires:
* building/eave height measured from building & canopy height raster
* pano id of corresponding building (to allow searching for pano image)
* interactively measured building/eave height in pixels
* interactively measured floor height in pixels  

Update the first two parameters each time and re-run all cells.

### Load dependencies

In [105]:
import cv2
import glob
import os

### Set input parameters:
* input pano images folder
* pano image id
* building/eave height (m) - derived from building/canopy height raster

In [106]:
pano_folder=r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\Pano_clipped_all"


In [107]:
# pano_id='Sdp4ax7QS6z-NBY_iBH5ww' # please update every time you run this notebook
# building_height_m=3.5 # remember to update correspondingly the same feature you are measuring

pano_id='qRwIoP-OssOI6-Czcvn9Xg'
building_height_m=3.5

In [108]:
# Load image
pano_file=glob.glob(os.path.join(pano_folder,pano_id+'.jpg'))[0]
original_img = cv2.imread(pano_file)

### Store measured points

In [109]:
# List to store two points
points_buildingheight = []
points_floorheight = []
dist_buildingheight=[]
dist_floorheight=[]


### defines callback function

In [110]:
def measure_distance(event, x, y, flags, param):
    """Handles mouse clicks to measure X and Y distances."""
    img_ref, points, dists, window_name = param  # Unpack parameters
    # global points, img

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))

        if len(points) == 2:
            # Calculate distances
            x_dist = abs(points[1][0] - points[0][0])
            y_dist = abs(points[1][1] - points[0][1])

            print(f"Start point coordinates: {points[0]}")
            print(f"End point coordinates: {points[1]}")
            print(f"X Distance: {x_dist} pixels")
            print(f"Y Distance: {y_dist} pixels")
            
            dists.append((x_dist,y_dist))

            # Draw points and line
            cv2.circle(img_ref, points[0], 5, (0, 0, 255), -1)  # Red point
            cv2.circle(img_ref, points[1], 5, (0, 255, 0), -1)  # Green point
            cv2.line(img_ref, points[0], (points[1][0], points[0][1]), (255, 0, 0), 2)  # Horizontal line
            cv2.line(img_ref, points[1], (points[1][0], points[0][1]), (255, 0, 0), 2)  # Vertical line
            # Refresh the window with updated image
            cv2.imshow(window_name, img_ref)

            points = []  # Reset points
            dists = []

### Interactively measure building/eave and floor heights
* two image windows will be open, one for building/eave height and another for floor height measuring
* single click on two points on each image to measure
* press esc key to finish

In [111]:
# Create two windows
window_buildingheight='Measure_building_height_'+pano_id
window_floorheight='Measure_floor_height_'+pano_id
cv2.namedWindow(window_buildingheight)
cv2.namedWindow(window_floorheight)

# Create copies of the original image for drawing
img_building = original_img.copy()
img_floor = original_img.copy()

# set mouse callback
cv2.setMouseCallback(window_buildingheight, measure_distance, param=(img_building, points_buildingheight, dist_buildingheight,window_buildingheight))
cv2.setMouseCallback(window_floorheight, measure_distance, param=(img_floor, points_floorheight, dist_floorheight,window_floorheight))

# Track window status
building_open = True
floor_open = True
# Show images and handle closing
while building_open or floor_open:
    # Only display open windows
    if building_open and cv2.getWindowProperty(window_buildingheight, cv2.WND_PROP_VISIBLE) >= 1:
        cv2.imshow(window_buildingheight, img_building)
    else:
        building_open = False  # Window is closed manually

    if floor_open and cv2.getWindowProperty(window_floorheight, cv2.WND_PROP_VISIBLE) >= 1:
        cv2.imshow(window_floorheight, img_floor)
    else:
        floor_open = False  # Window is closed manually

    key = cv2.waitKey(1) & 0xFF  # Check for key press

    # Close windows independently when ESC is pressed
    if key == 27:
        if floor_open:  # Close floor window first if open
            cv2.destroyWindow(window_floorheight)
            floor_open = False
        elif building_open:  # Then close building window if open
            cv2.destroyWindow(window_buildingheight)
            building_open = False

# Ensure all windows are closed
cv2.destroyAllWindows()

# while True:
#     # Display the windows
#     cv2.imshow(window_buildingheight, original_img)
#     cv2.imshow(window_floorheight, original_img)
#     # Wait for a key press
#     key = cv2.waitKey(0) & 0xFF  # Wait indefinitely until a key is pressed
#     # Check if ESC key is pressed (27 is the ASCII value of ESC)
#     if key == 27:  # ESC key
#         cv2.destroyWindow(window_buildingheight)  # Close the first window
#         break  # Exit the loop when the first window is closed

# while True:
#     key = cv2.waitKey(0) & 0xFF
#     if key == 27:  # ESC key
#         cv2.destroyWindow(window_buildingheight)
#         break

# # Close all windows when both are closed
# cv2.destroyAllWindows()

Start point coordinates: (1452, 2254)
End point coordinates: (1452, 2309)
X Distance: 0 pixels
Y Distance: 55 pixels
Start point coordinates: (1517, 1975)
End point coordinates: (1517, 2306)
X Distance: 0 pixels
Y Distance: 331 pixels


### Estimate floor height
* EEH estimation result will be printed out in the end

In [112]:
floor_height_m=dist_floorheight[0][1]/dist_buildingheight[0][1]*building_height_m
print('Estimated floor height (m):',floor_height_m)

Estimated floor height (m): 0.581570996978852
